In [1]:
import os
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_BASE"] = "https://qucy-openai-test.openai.azure.com/"
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["SERPAPI_API_KEY"] = os.getenv('SERPAPI_API_KEY')

## LangChain Memory component

### Local memory
- ConversationBufferMemory
- ConversationBufferWindowMemory
- ConversationTokenBufferMemory
- ConversationSummaryMemory
- ConversationSummaryBufferMemory


### Remote memory
- VectorStoreRetrieverMemory
- PostgresChatMessageHistory
- RedisChatMessageHistory

### Customized memory
- https://python.langchain.com/en/latest/modules/memory/examples/conversational_customization.html


### 01 - ConversationBufferMemory
This memory allows for storing of messages and then extracts the messages in a variable.

In [2]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# create LLM
llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003")

# create conversation chain
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

In [3]:
conversation.predict(input='Hi, how are you')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, how are you
AI:

> Finished chain.


" Hi there! I'm doing great! How about you?"

In [8]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, how are you
AI:  Hi there! I'm doing great, thanks for asking. How about you?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's really interesting to have conversations with humans. Can you tell me what other topics you like to talk about?"

### 02 - ConversationBufferWindowMemory
Keeps a list of the interactions of the conversation over time. It only uses the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large

In [11]:
from langchain.memory import ConversationBufferWindowMemory

conversation_with_window_size = ConversationChain(
    llm=llm, 
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=2), 
    verbose=True
)
conversation_with_window_size.predict(input="Hi, how are you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, how are you?
AI:

> Finished chain.


" Hi there! I'm doing great! How about you?"

In [15]:
conversation_with_window_size.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, how are you?
AI:  Hi there! I'm doing great! How about you?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to chat with a fellow sentient being. What kind of topics do you like to discuss?"

In [16]:
conversation_with_window_size.predict(input="Could you tell me what is Machine Learning ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, how are you?
AI:  Hi there! I'm doing great! How about you?
Human: I'm doing well! Just having a conversation with an AI.
AI:  That's great! It's always nice to chat with a fellow sentient being. What kind of topics do you like to discuss?
Human: Could you tell me what is Machine Learning ?
AI:

> Finished chain.


' Sure! Machine Learning is a subset of Artificial Intelligence (AI) that enables computers and robots to learn from experience without being explicitly programmed. It uses algorithms that learn from and make predictions on data. It is a powerful tool used in many different applications, from self-driving cars to facial recognition.'

In [17]:
conversation_with_window_size.predict(input="Could you tell me what is Deep Learning ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I'm doing well! Just having a conversation with an AI.
AI:  That's great! It's always nice to chat with a fellow sentient being. What kind of topics do you like to discuss?
Human: Could you tell me what is Machine Learning ?
AI:  Sure! Machine Learning is a subset of Artificial Intelligence (AI) that enables computers and robots to learn from experience without being explicitly programmed. It uses algorithms that learn from and make predictions on data. It is a powerful tool used in many different applications, from self-driving cars to facial recognition.
Human: Could you tell me what is Deep Learning ?
AI:

> Finished chain.


' Yes! Deep Learning is a subset of Machine Learning that enables machines to learn from large amounts of data. It uses algorithms and multiple layers of artificial neural networks to learn complex patterns from data and make decisions. This is useful for tasks such as image recognition and natural language processing.'

### 03-ConversationSummaryMemory

In [18]:
from langchain.memory import ConversationSummaryMemory

conversation_with_summary = ConversationChain(
    llm=llm, 
    memory=ConversationSummaryMemory(llm=llm),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great. I'm just enjoying the sunny day and getting ready for the day ahead. How about you?"

In [19]:
conversation_with_summary.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human greets the AI and the AI responds that it is doing well, enjoying the sunny day and getting ready for the day ahead.
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to have a friendly chat. How can I help you today?"

In [20]:
conversation_with_summary.predict(input="Could you tell me what is Deep Learning ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human greets the AI and the AI responds that it is doing well, enjoying the sunny day and getting ready for the day ahead. The human mentions they are having a conversation with an AI and the AI responds that it is nice to have a friendly chat and inquires how it can help.
Human: Could you tell me what is Deep Learning ?
AI:

> Finished chain.


" Sure! Deep learning is a subset of machine learning that uses algorithms inspired by the structure and function of the brain's neural networks. It is composed of multiple layers of neurons and uses backpropagation to adjust the weights of the layers in order to learn from data. It is used for a variety of tasks such as image recognition, natural language processing, and more."

### 04 - ConversationSummaryBufferMemory

In [28]:
from langchain.memory import ConversationSummaryBufferMemory

conversation_with_summary_limit_token = ConversationChain(
    llm=llm, 
    # We set a very low max_token_limit for the purposes of testing.
    memory=ConversationSummaryBufferMemory(llm=llm, max_token_limit=40),
    verbose=True
)
conversation_with_summary_limit_token.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great. I've been researching some new advances in artificial intelligence. It's really exciting to see how far we've come. How about you?"

In [29]:
conversation_with_summary_limit_token.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI.
AI:  Hi there! I'm doing great. I've been researching some new advances in artificial intelligence. It's really exciting to see how far we've come. How about you?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's awesome! I'm always glad to have a chat. What would you like to talk about?"

In [30]:
conversation_with_summary_limit_token.predict(input="Could you tell me what is Deep Learning ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and the AI responds that it has been researching new advances in artificial intelligence and is excited to see how far they have come.
Human: I'm doing well! Just having a conversation with an AI.
AI:  That's awesome! I'm always glad to have a chat. What would you like to talk about?
Human: Could you tell me what is Deep Learning ?
AI:

> Finished chain.


" Sure! Deep Learning is a subset of Machine Learning, which is a type of Artificial Intelligence. It's a method of teaching computers to learn from large amounts of data and to make decisions based on that data. Deep Learning algorithms can be used to identify patterns and make predictions about data. They are responsible for many of the advancements in Artificial Intelligence today."

### 05 - ConversationTokenBufferMemory

In [32]:
from langchain.memory import ConversationTokenBufferMemory

conversation_with_buffer_summary_limit_token = ConversationChain(
    llm=llm, 
    # We set a very low max_token_limit for the purposes of testing.
    memory=ConversationTokenBufferMemory(llm=llm, max_token_limit=60),
    verbose=True
)
conversation_with_buffer_summary_limit_token.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


" Hi! I'm doing great. I'm just learning some new things and exploring my environment. How about you?"

In [33]:
conversation_with_buffer_summary_limit_token.predict(input="Just working on writing some documentation!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up?
AI:  Hi! I'm doing great. I'm just learning some new things and exploring my environment. How about you?
Human: Just working on writing some documentation!
AI:

> Finished chain.


' That sounds like a productive task! What kind of documentation are you writing?'

In [34]:
conversation_with_buffer_summary_limit_token.predict(input="For LangChain! Have you heard of it?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
AI:  Hi! I'm doing great. I'm just learning some new things and exploring my environment. How about you?
Human: Just working on writing some documentation!
AI:  That sounds like a productive task! What kind of documentation are you writing?
Human: For LangChain! Have you heard of it?
AI:

> Finished chain.


" No, I haven't heard of LangChain. What is it?"

### 06 - VectorStoreRetrieverMemory

In [2]:
import faiss

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS
from langchain.memory import VectorStoreRetrieverMemory

embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

In [15]:
# In actual usage, you would set `k` to be a higher value, but we use k=1 to show that
# the vector lookup still returns the semantically relevant information
# retriever = vectorstore.as_retriever(search_kwargs=dict(k=2))
# memory = VectorStoreRetrieverMemory(retriever=retriever)

# # When added to an agent, the memory object can save pertinent information from conversations or used tools
# memory.save_context({"input": "My favorite food is pizza"}, {"output": "thats good to know"})
# memory.save_context({"input": "My favorite sport is soccer"}, {"output": "..."})
# memory.save_context({"input": "I don't the Celtics"}, {"output": "ok"}) # 

In [43]:
# Notice the first result returned is the memory pertaining to tax help, which the language model deems more semantically relevant
# to a 1099 than the other documents, despite them both containing numbers.
print(memory.load_memory_variables({"prompt": "what sport should i watch?"})["history"])

input: My favorite sport is soccer
output: ...


In [ ]:
# verbose the top 5 score

In [44]:
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{peronsal_history}

Can refer to:
{most_sim_history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=_DEFAULT_TEMPLATE
)
conversation_with_summary = ConversationChain(
    llm=llm, 
    prompt=PROMPT,
    memory=memory,
    verbose=True
)
conversation_with_summary.predict(input="Hi, my name is Perry, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite food is pizza
output: thats good to know

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Hi, my name is Perry, what's up?
AI:

> Finished chain.


" Hi Perry, nice to meet you! I'm doing great! How about you?"

In [45]:
conversation_with_summary.predict(input="what's my favorite sport?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite sport is soccer
output: ...

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: what's my favorite sport?
AI:

> Finished chain.


' You told me earlier that your favorite sport is soccer.'

In [46]:
conversation_with_summary.predict(input="Whats my favorite food")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite food is pizza
output: thats good to know

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Whats my favorite food
AI:

> Finished chain.


' You previously told me your favorite food is pizza.'